# Exercicios com Spark

Caso precise, abaixo estão os comandos para iniciar o container:

Para macOS e linux, utilize:

```bash
docker run \
    -it \
    --rm \
    -p 8888:8888 \
    -p 4040:4040 \
    -v "`pwd`":/home/jovyan/work \
    jupyter/pyspark-notebook


```

Se estiver no Windows estes comandos, utilize:

- No Powershell: `docker run -it --rm -p 8888:8888 -p 4040:4040 -v ${PWD}:/home/jovyan/work jupyter/pyspark-notebook`

- No Prompt de comando: `docker run -it --rm -p 8888:8888 -p 4040:4040 -v %cd%:/home/jovyan/work jupyter/pyspark-notebook`

Agora abra esse notebook lá no container!


## Iniciando o Spark

In [1]:
import pyspark

conf = pyspark.SparkConf()
conf.setAppName("Minha aplicação")
conf.setMaster("local[*]")

sc = pyspark.SparkContext(conf=conf)

In [2]:
sc

<SparkContext master=local[*] appName=Minha aplicação>

## Iniciando a biblioteca de correção

In [5]:
# !pip install git+https://github.com/macielcalebe/insperautograding.git

In [6]:
import insperautograder.jupyter as ia
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
ia.tasks()

|    | Atividade            | De                  | Até                 | Conta como ATV?   | % Nota Atraso   |
|---:|:---------------------|:--------------------|:--------------------|:------------------|:----------------|
|  0 | newborn              | 2025-02-01 00:00:00 | 2025-05-30 00:00:00 | Não               | 0%              |
|  1 | select01             | 2025-02-05 00:00:00 | 2025-02-15 23:59:59 | Sim               | 25%             |
|  2 | ddl                  | 2025-02-19 00:00:00 | 2025-02-26 23:59:59 | Sim               | 25%             |
|  3 | dml                  | 2025-02-24 00:00:00 | 2025-03-09 23:59:59 | Sim               | 25%             |
|  4 | agg_join             | 2025-02-26 00:00:00 | 2025-03-09 23:59:59 | Sim               | 25%             |
|  5 | group_having         | 2025-03-10 00:00:00 | 2025-03-17 23:59:59 | Sim               | 25%             |
|  6 | views                | 2025-03-12 00:00:00 | 2025-03-19 23:59:59 | Sim               | 25%             |
|  7 | sql_review1          | 2025-03-17 00:00:00 | 2025-03-23 23:59:59 | Sim               | 25%             |
|  8 | ai_md_23_2           | 2025-03-23 00:00:00 | 2025-03-30 23:59:59 | Sim               | 25%             |
|  9 | ai_md_23_1           | 2025-03-23 00:00:00 | 2025-03-30 23:59:59 | Sim               | 25%             |
| 10 | permissions          | 2025-03-24 00:00:00 | 2025-04-08 23:59:59 | Sim               | 25%             |
| 11 | ai_md_25_1           | 2025-03-31 00:00:00 | 2025-03-31 10:05:00 | Não               | 0%              |
| 12 | desafio_normalizacao | 2025-04-07 00:00:00 | 2025-05-30 00:00:00 | Não               | 0%              |
| 13 | triggers             | 2025-04-23 00:00:00 | 2025-05-04 23:59:59 | Sim               | 25%             |
| 14 | functional           | 2025-05-05 00:00:00 | 2025-05-11 23:59:59 | Sim               | 25%             |
| 15 | spark                | 2025-05-07 00:00:00 | 2025-05-14 23:59:59 | Sim               | 25%             |
| 16 | exercicios_spark     | 2025-05-12 00:00:00 | 2025-05-18 23:59:59 | Sim               | 25%             |

In [8]:
ia.grades(task="exercicios_spark")

|    | Atividade        | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:-----------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | exercicios_spark | ex01        |      1 |     10 |                10 |                 0 |
|  1 | exercicios_spark | ex02        |      1 |     10 |                10 |                 0 |
|  2 | exercicios_spark | ex03        |      1 |     10 |                10 |                 0 |
|  3 | exercicios_spark | ex04        |      1 |     10 |                10 |                 0 |
|  4 | exercicios_spark | ex05        |      1 |      0 |                 0 |                 0 |
|  5 | exercicios_spark | ex06        |      1 |      0 |                 0 |                 0 |

## Trabalhando com Spark

Para este exercicio vamos trabalhar com o dataset de reviews da Amazon visto em https://www.kaggle.com/datasets/kritanjalijain/amazon-reviews.

Baixe o arquivo "train.csv" direto do Kaggle, ou pelo link https://bigdata-22-2.s3.us-east-2.amazonaws.com/amazon_sentiment/train.csv.gz

Vamos ler o arquivo "train.csv" em um RDD.

In [9]:
rdd = sc.textFile("train.csv")

In [10]:
rdd.take(1)

['"2","Stuning even for the non-gamer","This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^"']

De acordo com a documentação deste arquivo vista no Kaggle, cada linha contem 2 elementos: o sentimento do review (1 - negativo, 2 - positivo), o título e o corpo do review. A linha contem esses elementos em um formato "comma-separated value" (CSV), onde cada um dos campos está delimitado por aspas duplas. Se o texto em si (titulo ou corpo) contem aspas, elas aparecem como um par de aspas duplas. Vamos usar o `.filter()` para achar um exemplo desses.

In [11]:
example_line = rdd.filter(lambda x: '""' in x).take(1)
example_line = example_line[0]

example_line

'"2","Amazing!","This soundtrack is my favorite music of all time, hands down. The intense sadness of ""Prisoners of Fate"" (which means all the more if you\'ve played the game) and the hope in ""A Distant Promise"" and ""Girl who Stole the Star"" have been an important inspiration to me personally throughout my teen years. The higher energy tracks like ""Chrono Cross ~ Time\'s Scar~"", ""Time of the Dreamwatch"", and ""Chronomantique"" (indefinably remeniscent of Chrono Trigger) are all absolutely superb as well.This soundtrack is amazing music, probably the best of this composer\'s work (I haven\'t heard the Xenogears soundtrack, so I can\'t say for sure), and even if you\'ve never played the game, it would be worth twice the price to buy it.I wish I could give it 6 stars."'

Levando isso em consideração, vamos fazer uma função simples para separar os campos:

In [12]:
def parse_line(line):
    parts = line[1:-1].split('","')
    sentiment = int(parts[0])
    title = parts[1].replace('""', '"')
    body = parts[2].replace('""', '"')
    return (sentiment, title, body)

In [13]:
parse_line(example_line)

(2,
 'Amazing!',
 'This soundtrack is my favorite music of all time, hands down. The intense sadness of "Prisoners of Fate" (which means all the more if you\'ve played the game) and the hope in "A Distant Promise" and "Girl who Stole the Star" have been an important inspiration to me personally throughout my teen years. The higher energy tracks like "Chrono Cross ~ Time\'s Scar~", "Time of the Dreamwatch", and "Chronomantique" (indefinably remeniscent of Chrono Trigger) are all absolutely superb as well.This soundtrack is amazing music, probably the best of this composer\'s work (I haven\'t heard the Xenogears soundtrack, so I can\'t say for sure), and even if you\'ve never played the game, it would be worth twice the price to buy it.I wish I could give it 6 stars.')

Podemos agora utilizar nossa função para separar os campos de cada linha do dataset. 

In [14]:
rdd_split = rdd.map(parse_line)

Como de costume, nada realmente acontece até que uma "action" seja invocada. O `.map()` é uma "transformation". Vamos usar uma action simples para "materializar" o novo RDD.

In [15]:
rdd_split.count()

3600000

Vamos explorar os resultados para ver se deu certo

In [16]:
rdd_split.take(1)

[(2,
  'Stuning even for the non-gamer',
  'This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^')]

**Atividade**: Implemente uma função que recebe o rdd processado e conte quantos sentimentos diferentes existem, e quantas vezes aparecem, para confirmar que só tem os sentimentos 1 e 2. Sua função deve retornar o resultado em tuplas, onde o primeiro elemento é o sentimento e o segundo é a contagem de vezes que aparece.

In [14]:
def ex01(rdd_split):
    rdd_sentiment = rdd_split.map(lambda x: (x[0], 1))\
                            .reduceByKey(lambda x, y: x+y)
    return rdd_sentiment.collect() # Pode alterar o retorno, se necessário!
    
ex01(rdd_split)

[(1, 1800000), (2, 1800000)]

In [15]:
ia.sender(answer="ex01", task="exercicios_spark", question="ex01", answer_type="pycode")

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Atividade**: Implemente uma função que recebe o rdd processado e retorna quantos reviews não tem titulo.

In [15]:
def ex02(rdd_split):
    rdd_no_title = rdd_split.filter(lambda x: x[1] == '').count()
    return rdd_no_title # Pode alterar o retorno, se necessário!

ex02(rdd_split)

48

In [29]:
ia.sender(answer="ex02", task="exercicios_spark", question="ex02", answer_type="pycode")

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Atividade**: Implemente uma função que recebe o rdd processado e retorna quantos reviews não tem corpo.

In [16]:
def ex03(rdd_split):
    rdd_no_body = rdd_split.filter(lambda x: x[2] == '').count()
    return rdd_no_body # Pode alterar o retorno, se necessário!

ex03(rdd_split)

0

In [31]:
ia.sender(answer="ex03", task="exercicios_spark", question="ex03", answer_type="pycode")

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Atividade**: Implemente uma função que recebe o rdd processado e retorna qual o comprimento máximo de um título e de um corpo. O resultado deve ser uma tupla com os dois valores.

In [17]:
def ex04(rdd_split):
    max_title_size = rdd_split.map(lambda x: len(x[1]))\
                            .takeOrdered(1, lambda x: -x)[0]

    max_body_size = rdd_split.map(lambda x: len(x[2]))\
                            .takeOrdered(1, lambda x: -x)[0]
    
    return (max_title_size, max_body_size) # Pode alterar o retorno, se necessário!
    
ex04(rdd_split)

(139, 1010)

In [39]:
ia.sender(answer="ex04", task="exercicios_spark", question="ex04", answer_type="pycode")

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Atividade**: Implemente uma função que recebe o rdd processado e retorna qual a maior palavra palíndroma sem pontuações do dataset (no titulo ou corpo) e seu tamanho.

Para este exercício, está permitido o uso de list comprehensions.

**Obs**: Após o split, se uma palavra tiver alguma `string.punctuation`, ela deve ser desconsiderada (ao invés de remover as pontuações da palavra, remova as palavras com alguma pontuação).

In [18]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
def ex05(rdd_split):
    def ha_pontuacao(palavra):
        for char in palavra:
            if char in string.punctuation:
                return False
        return True

    palavras = rdd_split.map(lambda x: x[1] + ' ' + x[2])

    palindromos = palavras.flatMap(lambda x: x.split())\
                        .filter(lambda x: x == x[::-1])\
                        .filter(ha_pontuacao)

    maior_palindromo = palindromos.takeOrdered(1, lambda x: -len(x))[0]
    
    return (maior_palindromo, len(maior_palindromo)) # Pode alterar o retorno, se necessário!

ex05(rdd_split)

('lllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll', 63)

In [22]:
ia.sender(answer="ex05", task="exercicios_spark", question="ex05", answer_type="pycode")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Atividade**: Implemente uma função que recebe o rdd processado e retorna as 20 palavras mais populares do titulo com sua frequência absoluta. Teste no subconjunto apresentado abaixo.

In [23]:
def ex06(rdd_split):
    mais_populares = rdd_split.flatMap(lambda x: x[1].split())\
                            .map(lambda x: (x, 1))\
                            .reduceByKey(lambda x, y: x+y)\
                            .takeOrdered(20, lambda x: -x[1])

    return mais_populares # Pode alterar o retorno, se necessário!

ex06(rdd_split)

[('the', 348807),
 ('a', 249841),
 ('of', 241846),
 ('for', 220429),
 ('and', 190973),
 ('to', 177502),
 ('A', 173889),
 ('Great', 168288),
 ('I', 145120),
 ('is', 143982),
 ('Not', 140413),
 ('not', 128331),
 ('this', 121247),
 ('The', 119549),
 ('it', 107117),
 ('but', 95869),
 ('book', 95629),
 ('good', 87873),
 ('Good', 86357),
 ('in', 84964)]

In [24]:
rdd_redux = rdd_split.sample(False, 0.05, 7)
ex06(rdd_redux)

[('the', 17380),
 ('a', 12537),
 ('of', 11944),
 ('for', 11143),
 ('and', 9590),
 ('to', 8853),
 ('A', 8536),
 ('Great', 8475),
 ('is', 7305),
 ('I', 7187),
 ('Not', 6953),
 ('not', 6424),
 ('this', 5995),
 ('The', 5933),
 ('it', 5349),
 ('book', 4774),
 ('but', 4693),
 ('good', 4404),
 ('in', 4313),
 ('Good', 4295)]

In [25]:
ia.sender(answer="ex06", task="exercicios_spark", question="ex06", answer_type="pycode")

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Conferir notas

In [26]:
ia.tasks()

|    | Atividade            | De                  | Até                 | Conta como ATV?   | % Nota Atraso   |
|---:|:---------------------|:--------------------|:--------------------|:------------------|:----------------|
|  0 | newborn              | 2025-02-01 00:00:00 | 2025-05-30 00:00:00 | Não               | 0%              |
|  1 | select01             | 2025-02-05 00:00:00 | 2025-02-15 23:59:59 | Sim               | 25%             |
|  2 | ddl                  | 2025-02-19 00:00:00 | 2025-02-26 23:59:59 | Sim               | 25%             |
|  3 | dml                  | 2025-02-24 00:00:00 | 2025-03-09 23:59:59 | Sim               | 25%             |
|  4 | agg_join             | 2025-02-26 00:00:00 | 2025-03-09 23:59:59 | Sim               | 25%             |
|  5 | group_having         | 2025-03-10 00:00:00 | 2025-03-17 23:59:59 | Sim               | 25%             |
|  6 | views                | 2025-03-12 00:00:00 | 2025-03-19 23:59:59 | Sim               | 25%             |
|  7 | sql_review1          | 2025-03-17 00:00:00 | 2025-03-23 23:59:59 | Sim               | 25%             |
|  8 | ai_md_23_2           | 2025-03-23 00:00:00 | 2025-03-30 23:59:59 | Sim               | 25%             |
|  9 | ai_md_23_1           | 2025-03-23 00:00:00 | 2025-03-30 23:59:59 | Sim               | 25%             |
| 10 | permissions          | 2025-03-24 00:00:00 | 2025-04-08 23:59:59 | Sim               | 25%             |
| 11 | ai_md_25_1           | 2025-03-31 00:00:00 | 2025-03-31 10:05:00 | Não               | 0%              |
| 12 | desafio_normalizacao | 2025-04-07 00:00:00 | 2025-05-30 00:00:00 | Não               | 0%              |
| 13 | triggers             | 2025-04-23 00:00:00 | 2025-05-04 23:59:59 | Sim               | 25%             |
| 14 | functional           | 2025-05-05 00:00:00 | 2025-05-11 23:59:59 | Sim               | 25%             |
| 15 | spark                | 2025-05-07 00:00:00 | 2025-05-14 23:59:59 | Sim               | 25%             |
| 16 | exercicios_spark     | 2025-05-12 00:00:00 | 2025-05-18 23:59:59 | Sim               | 25%             |

In [27]:
ia.grades(task="exercicios_spark")

|    | Atividade        | Exercício   |   Peso |   Nota |   Nota Sem Atraso |   Nota Com Atraso |
|---:|:-----------------|:------------|-------:|-------:|------------------:|------------------:|
|  0 | exercicios_spark | ex01        |      1 |     10 |                10 |                 0 |
|  1 | exercicios_spark | ex02        |      1 |     10 |                10 |                 0 |
|  2 | exercicios_spark | ex03        |      1 |     10 |                10 |                 0 |
|  3 | exercicios_spark | ex04        |      1 |     10 |                10 |                 0 |
|  4 | exercicios_spark | ex05        |      1 |     10 |                10 |                 0 |
|  5 | exercicios_spark | ex06        |      1 |     10 |                10 |                 0 |